In [1]:
import numpy as np

In [71]:
def gram_matrix(in_points, origin=[0,0,0]):
    points = []
    N = len(in_points)
    for p in in_points: 
        points.append(p-origin)
#     print(points)
    G = np.zeros((N,N))
    for i in range(N):
        for j in range(N):
            G[i, j] = np.dot(points[i], points[j])
    return G, points


def distance_matrix(in_points, origin=[0,0,0]):
    points = []
    N = len(in_points)
    for p in in_points: 
        points.append(p-origin)
        
    D = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
                D[i, j] = np.linalg.norm(points[i] - points[j])
            
    return D, points

def gram_to_dmat(G):
    N = G.shape[0]
    distances = np.zeros((N, N))
    for i in range(N):
        for j in range(i+1, N):
            distances[i,j] = np.sqrt(G[i,i] + G[j,j] - 2*G[i,j])
            distances[j,i] = distances[i,j]
    return distances


def points_from_gram_matrix(G, origin=[0,0,0]):
    #not sure about this 
    N = G.shape[0]
    eigenvalues, eigenvectors = np.linalg.eigh(G)
    points = np.zeros((N, 3))
    for i in range(3):
        points[:, i] = np.sqrt(eigenvalues[i]) * eigenvectors[:, i]
        points[:, i][:3]+= origin
#         print(points[:,i].shape)
    return points


$$ d_{ij}^2 = (v_{io}-v_{jo}).(v_{io}-v_{jo})$$
where $o$ is the origin.
But $(v_{io}-v_{jo})$ = $(v_{i}-v_{j})$, so the distance matrix is independent of origin as it should be. 

Also notice that if we denote the row and column mean by r and c respectively,
$$ r = \frac{1}{N}\sum_{j=1}^N d_{ij}^2 = v_{i}^2 + \frac{1}{N} \sum_{j=1}^N  v_{j}^2 - 2 v_{i}.v_{j} $$
$$ c = \frac{1}{N}\sum_{i=1}^N d_{ij}^2 = v_{j}^2 + \frac{1}{N} \sum_{i=1}^N  v_{i}^2 - 2 v_{i}.v_{j} $$
and denote the mean over all elements of the matrix as m 
$$m = \frac{1}{N}\sum_{i,j=1}^N d_{ij}^2 = \frac{1}{N} \sum_{i,j=1}^N  v_{i}^2 +v_{j}^2  - 2 v_{i}.v_{j}$$

Then we notice that 
$$r+c-m = v_{i}^2  + v_{j}^2$$
This means that subtracting r+c-m (computed for each element), from the element leaves us with $2 v_{i}.v_{j}$
Below we use this to extract the product (but we lost info about the origin. )

In [68]:
def dmat_to_gram(D):
    #this loses the info on the origin of the gram matrix 
    
    D2 = D**2
    N = D.shape[0]
    gram = np.zeros((N, N))
    matrix_mean = np.mean(D2)
    for i in range(N):
        row_mean = np.mean(D2[i,:])
        for j in range(i, N):
            col_mean = np.mean(D2[:,j])
            gram[i,j] = -(D2[i,j] - (row_mean+col_mean-matrix_mean ))/2
            gram[j,i]=gram[i,j]
    return gram

A possible way to add info about the origin 
$$<v_{io}| v_{jo}> = <v_i-v_o| v_{j}-v_o> = <v_i|v_j> +<v_o|v_o> -<v_i|v_o> -<v_j|v_o> $$

should check this 


In [62]:
points = np.asarray([[1, 5, 3], [4,5,6], [7, 8, 9], [10, 1, 2]])

In [63]:
origin0=[0,0,0]
G0, pg0 = gram_matrix(points, origin0 )
D0, pd0 = distance_matrix(points, origin0)

In [69]:
dmat_to_gram(D0)

array([[ 24.3125,   4.8125, -13.9375, -15.1875],
       [  4.8125,   3.3125,   2.5625, -10.6875],
       [-13.9375,   2.5625,  28.8125, -17.4375],
       [-15.1875, -10.6875, -17.4375,  43.3125]])

In [70]:
G0

array([[ 35.,  47.,  74.,  21.],
       [ 47.,  77., 122.,  57.],
       [ 74., 122., 194.,  96.],
       [ 21.,  57.,  96., 105.]])

In [6]:
origin1=[0,0,1]
G1, pg1 = gram_matrix(points, origin1)
D1, pd1 = distance_matrix(points, origin1)

In [7]:
origin2=[1,2,3]
G2, pg2 = gram_matrix(points, origin2 )
D2, pd2 = distance_matrix(points, origin2)

In [8]:
# D1 - D2
# D1 - D3

In [10]:
gram_to_dmat(G1) - D1

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [10]:
eval0, evec0 = np.linalg.eigh(G0)
eval1, evec1 = np.linalg.eigh(G1)
eval2, evec2 = np.linalg.eigh(G2)

In [11]:
points_from_gram_matrix(G1, origin1)

array([[-7.98393818e-22, -1.73783970e+00,  2.64666478e+00],
       [-1.42545419e-07,  3.92637474e-01,  1.53935705e+00],
       [ 1.00000009e+00,  1.59568479e+00,  3.00869668e+00],
       [-5.85803090e-09, -4.20345076e-01, -6.65767238e+00]])